In [1]:
import numpy as np
import keras
import sklearn
import matplotlib.pyplot as plt 
from sklearn.datasets import make_classification
import math

# Quantum libraries

import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer, QNGOptimizer, RotosolveOptimizer


2023-11-03 20:11:35.673968: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-03 20:11:35.683827: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 20:11:35.770975: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 20:11:35.771035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 20:11:35.771193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [12]:
def reshape_x(input_array):
    # Calculate the number of subarrays needed
    num_subarrays = len(input_array) // 3 + (len(input_array) % 3 != 0)

    # Split the array into subarrays of size 3
    subarrays = [input_array[i * 3:(i + 1) * 3] for i in range(num_subarrays)]

    # Check if the last subarray has fewer than 3 elements and pad with zeros if necessary
    if len(subarrays[-1]) < 3:
        subarrays[-1] = np.pad(subarrays[-1], (0, 3 - len(subarrays[-1])), mode='constant')
        
    return subarrays

dev = qml.device("lightning.gpu", wires=1)
# Install any pennylane-plugin to run on some particular backend

@qml.qnode(dev, interface="autograd")
def qcircuit_basic(params, x, y):
    """A variational quantum circuit representing the Universal classifier.
    This version follows the Pennylane demo and is a simplified implemenation. 

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    for p in params: # Iterate num_layers times
        for x_sub in x:
            qml.Rot(*x_sub, wires=0)
        qml.Rot(*p, wires=0)
    return qml.expval(qml.Hermitian(y, wires=[0]))

@qml.qnode(dev, interface="autograd")
def qcircuit(params, x, y):
    """A variational quantum circuit representing the Universal classifier.
    This version follows the description of the paper to "incorporate data and processing of angles in a single step"
    
    Args:
        params (array[float]): array of parameters of dim ( num_layers, 2, ceil(len(x)/3)*3 ) 
        x (array[float]): single input vector 
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    
    for l in range(params.shape[0]): # Iterate num_layers times
        w = params[l, 0]
        b = params[l, 1]
        
        # print('x: ', x)
        # print('x.shape: ', x.shape)
        # print('w.shape: ', w.shape)
        # print('b.shape: ', b.shape)
        
        weighted_sum = w * x + b
        
        weighted_sum = reshape_x(weighted_sum)
        
        for x_sub in weighted_sum:
            qml.Rot(*x_sub, wires=0)
    
    return qml.expval(qml.Hermitian(y, wires=[0]))


def cost(params, x, y, state_labels=None):
    """Cost function to be minimized.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): array of state representations for labels

    Returns:
        float: loss value to be minimized
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        f = qcircuit(params, x[i], dm_labels[y[i]])
        loss = loss + (1 - f) ** 2
    return loss / len(x)

### Utility functions for testing and creating batches



In [13]:
def test(params, x, y, state_labels=None):
    """
    Tests on a given set of data.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        predicted (array([int]): predicted labels for test data
        output_states (array[float]): output quantum states from the circuit
    """
    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        fidel_function = lambda y: qcircuit(params, x[i], y)
        fidelities = [fidel_function(dm) for dm in dm_labels]
        best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(predicted), np.array(fidelity_values)


def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    score = y_true == y_pred
    return score.sum() / len(y_true)


def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data

    Args:
        inputs (array[float]): input data
        targets (array[float]): targets

    Returns:
        inputs (array[float]): one batch of input data of length `batch_size`
        targets (array[float]): one batch of targets of length `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [14]:
# Define output labels as quantum state vectors for binary classification

def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T


label_0 = [[1], [0]] # Digit Zero 
label_1 = [[0], [1]] # Digit One
state_labels = np.array([label_0, label_1], requires_grad=False)

In [20]:
def train_q_classifier(n_samples = 500, n_features = 6, n_informative = 2, n_classes = 2, 
                num_layers = 10, learning_rate = 0.6, epochs = 10, batch_size = 32):
    
    # n_samples = 500
    # n_features = 6
    # n_informative = 2
    # n_redundant = 4
    # n_classes = 2

    # Create a synthetic dataset with 2 classes
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_informative,
        n_redundant=(n_features - n_informative),
        n_classes=n_classes,
        class_sep=2.,
        random_state=0
    )
    
    # X = np.array(X)
    # y = np.array(y)

    # set training and test data
    split = int(0.8 * n_samples)
    X_train = X[:split, :]
    y_train = y[:split]
    X_test = X[split:, :]
    y_test = y[split:]

    # Train using Adam optimizer and evaluate the classifier
    # num_layers = 10
    # learning_rate = 0.6
    # epochs = 10
    # batch_size = 32

    opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)
    # opt = RotosolveOptimizer()

    # initialize random weights
    params = np.random.uniform(size=(num_layers, 2, n_features), requires_grad=True)

    label_0 = [[1], [0]] # Digit Zero 
    label_1 = [[0], [1]] # Digit One
    state_labels = np.array([label_0, label_1], requires_grad=False)

    predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
    accuracy_train = accuracy_score(y_train, predicted_train)

    predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
    accuracy_test = accuracy_score(y_test, predicted_test)

    # save predictions with random weights for comparison
    initial_predictions = predicted_test

    loss = cost(params, X_test, y_test, state_labels)

    print(
        "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
            0, loss, accuracy_train, accuracy_test
        )
    )

    for it in range(epochs):
        for Xbatch, ybatch in iterate_minibatches(X_train, y_train, batch_size=batch_size):
            params, _, _, _ = opt.step(cost, params, Xbatch, ybatch, state_labels)

        predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
        accuracy_train = accuracy_score(y_train, predicted_train)
        loss = cost(params, X_train, y_train, state_labels)

        predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
        accuracy_test = accuracy_score(y_test, predicted_test)
        res = [it + 1, loss, accuracy_train, accuracy_test]
        print(
            "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
                *res
            )
        )
        
    return float(accuracy_test)

In [22]:
_ = train_q_classifier(n_features=3, n_informative=3, num_layers=10)

Epoch:  0 | Cost: 0.277634 | Train accuracy: 0.585000 | Test Accuracy: 0.570000
Epoch:  1 | Loss: 0.321084 | Train accuracy: 0.525000 | Test accuracy: 0.560000
Epoch:  2 | Loss: 0.326920 | Train accuracy: 0.487500 | Test accuracy: 0.540000
Epoch:  3 | Loss: 0.316163 | Train accuracy: 0.547500 | Test accuracy: 0.620000
Epoch:  4 | Loss: 0.307821 | Train accuracy: 0.540000 | Test accuracy: 0.470000
Epoch:  5 | Loss: 0.306118 | Train accuracy: 0.522500 | Test accuracy: 0.490000
Epoch:  6 | Loss: 0.351159 | Train accuracy: 0.500000 | Test accuracy: 0.470000
Epoch:  7 | Loss: 0.330241 | Train accuracy: 0.495000 | Test accuracy: 0.470000
Epoch:  8 | Loss: 0.287979 | Train accuracy: 0.582500 | Test accuracy: 0.510000
Epoch:  9 | Loss: 0.308528 | Train accuracy: 0.527500 | Test accuracy: 0.530000
Epoch: 10 | Loss: 0.327481 | Train accuracy: 0.500000 | Test accuracy: 0.550000


### Analysis
This quantum classifier with a single qubit is found to be ineffective for input with dimension greater than 6. The accuracy is just around 50%.
A multiqubit classifier should be tested next. 